# Lab8

---

## Task

Реализовать решение ОДУ сеточным методом.

- Реализовать один из проекционных методов: метод Ритца или метод Галеркина.
- Сравнить решения при разных N (либо графически, либо выводить значения решений на достаточно частой сетке)

# Solution

---

In [ ]:
import math
import numpy as np
from scipy import linalg as la
from scipy import special as sp
from scipy import misc
from scipy import integrate

import matplotlib.pyplot as plt

### Реализация метода Галеркина

In [ ]:
def jacobi_polynom(n, k):
    return lambda x: (1 - x ** 2) * sp.eval_jacobi(n, k, k, x)

def derivative(func, order):
    return lambda x: misc.derivative(func, x, n=order)

def galerkin_method(p, q, r, f, segment, n):
    a, b = segment

    A_i = lambda phi: lambda x: p(x) * derivative(phi, 2)(x) + q(x) * derivative(phi, 1)(x) + r(x) * phi(x)
    A_i = np.vectorize(A_i)
    
    phi = [jacobi_polynom(i, 1) for i in range(n)]

    lbase = np.zeros( (n, n) )
    rbase = np.zeros( (n, 1) )

    for i in range(n):
        for j in range(n):
            lbase[i, j] = integrate.quad(lambda x: phi[i](x) * A_i(phi)[j](x), a, b)[0]
        rbase[i] = integrate.quad(lambda x: phi[i](x) * f(x), a, b)[0]

    alpha = la.solve(lbase, rbase)
    
    return lambda x: sum([alpha[i] * phi[i](x) for i in range(n)])


## Experimental reseach

In [ ]:
def experiment(p, q, r, f, segment, n_range):
    _, axes = plt.subplots(3, 2, figsize=(20, 12))

    k = 0
    for i in range(2):
        for j in range(2):
            n = n_range[k]
            k += 1
            
            a, b = segment
            u = galerkin_method(p, q, r, f, segment, n)
            grid = np.linspace(a, b, 100)

            axes[i, j].plot(grid, u(grid))
            axes[i, j].set_title(f"Number of N: {n}")
    
    for n in n_range:
        u = galerkin_method(p, q, r, f, segment, n)
        axes[2, 0].plot(grid, u(grid), label=f"N = {n}")

    axes[2, 0].legend()


### Дифуры второго порядка из методички

In [ ]:
# Var 5
p = lambda x: - 1 / (x + 3)
q = lambda x: - x
r = lambda x: math.log(2 + x)
f = lambda x: 1 - x / 2

segment = (-1, 1)
experiment(p, q, r, f, segment, [2, 5, 8, 11])

In [ ]:
# Var 8
p = lambda x: -(4 - x) / (5 - 2 * x)
q = lambda x: (1 - x)/2
r = lambda x: math.log(3 + x) / 2
f = lambda x: 1 + x / 3

segment = (-1, 1)
experiment(p, q, r, f, segment, [2, 5, 8, 11])

In [ ]:
# Var 11
p = lambda x: -(7 - x)/(8 + 3 * x)
q = lambda x: (1 + x / 3)
r = lambda x: (1 - math.exp(x / 2) / 2)
f = lambda x: 1/2 - x / 3 

segment = (-1, 1)
experiment(p, q, r, f, segment, [2, 5, 8, 11])

### Дифур первого порядка


In [ ]:
p = lambda x: 0
q = lambda x: 1 / (2 + x)
r = lambda x: math.cos(x)
f = lambda x: 1 + x
segment = (-1, 1)
experiment(p, q, r, f, segment, [2, 5, 8, 11])